In [1]:
from importlib import reload
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from transformers import (
    AutoConfig, AutoModelForCausalLM, AutoTokenizer
)

pl.seed_everything(42)

/home/soroosh/projects/general_env/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(
Global seed set to 42


42

## tokenizer and dataset

In [2]:
name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(name)

tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>', 
    "pad_token": '<pad>',
    "unk_token": '<unk>',
})

0

In [3]:
import src.data
reload(src.data)
from src.data import PoemDataset, get_dataloaders

dataset = PoemDataset(tokenizer, 'data/all_poems.json', max_len=512)
len(dataset)

11579631

In [4]:
tl, vl = get_dataloaders(dataset, val_frac=0.1, batch_size=5)

train dataset has 10421668 samples and val dataset has 1157963 samples


In [5]:
b = next(iter(tl))
for k in b:
    print(k, b[k].shape)

input_ids torch.Size([5, 177])
attention_mask torch.Size([5, 177])


## model

In [6]:
import src.model
reload(src.model)
from src.model import PoetFormer

model = PoetFormer(pretrained="HooshvareLab/gpt2-fa")

using pretrained model: HooshvareLab/gpt2-fa


In [7]:
model.count_parameters()

118099200

In [8]:
res = model.generate(num_return_sequences=1, max_length=128, n_beam=1)
for r in res:
    print(r)

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


حافظ:
وی «ان» هم که از این کلمه‌ها در ادبیات ما استفاده شده، این کلمه‌ها، به شکل «ان» هستند. حال، یک سوال مطرح می‌شود که چطور می‌توانیم این کلمات را در کلماتمان جای دهیم؟ این سوال، یکی از مسائل بسیار مهم در علم منطق است. «ان» در لغت به معنی «بودن» است و در واقع همان «بودن» است. «ان» در زبان فارسی به معنای «بودن» است و «بودن» به معنی «بودن» است. پس «بودن» یعنی «بودن». برای پاسخ به این سوال، فرض کنید می‌


## train

In [9]:
name = 'GPT2-fa-ganjoor-conditional'
print('model name:',name)

logger = TensorBoardLogger(save_dir='logs/', name=name)
lr_logger = LearningRateMonitor(logging_interval='step')
checkpoint = ModelCheckpoint(
    dirpath=f'weights/{name}/', 
    filename='{epoch}-{val_loss:.2f}', 
    monitor='val_loss',
    save_top_k=1, 
    period=1
)

trainer = pl.Trainer(
    benchmark=True, 
    gpus=1, 
    accumulate_grad_batches=1,
    logger=logger, 
    max_epochs=1,
    callbacks=[checkpoint, lr_logger]
)

/home/soroosh/projects/general_env/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:396: LightningDeprecationWarning: Argument `period` in `ModelCheckpoint` is deprecated in v1.3 and will be removed in v1.5. Please use `every_n_val_epochs` instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


model name: GPT2-fa-ganjoor-conditional


In [ ]:
trainer.fit(model, tl, vl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 118 M 
------------------------------------------
118 M     Trainable params
0         Non-trainable params
118 M     Total params
472.397   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint(f'weights/{name}/last.ckpt')

## generate

In [ ]:
res = model.generate(num_return_sequences=1, max_length=128, n_beam=1)
for r in res:
    print(r)